# P43 042：算术运算1

In [2]:
# ？？type是class类型的统称，包含如type、int
class A:
    pass
print(type(A),type(type))
print(type(1),type(int))

<class 'type'> <class 'type'>
<class 'int'> <class 'type'>


# P44 043：算术运算2

In [4]:
class NewInt(int):
    def __radd__(self,other):
        return int.__sub__(self,other)
a = NewInt(3)
print(a+1,1+a)  
# 1+a：1是int型，int中没有int和NewInt相加的__add__方法，故调用a的__radd__方法
# 注意1+a 是3-1=2 而不是1-3

4 2


# P45 044：计时器、__str__、__repr__、localtime

In [20]:
#  自制计时器
from datetime import datetime

class Timer:
    def __init__(self):
        self.status = "Stop"
        self.t1 = 0
        self.t2 = 0
    def start(self):
        if self.status == "Start":
            print("计时器已启动！")
        else:
            self.status = "Start"
            self.t1 = datetime.now()
    def stop(self):
        if self.status == "Stop":
            print("计时器已停止！")
        else:
            self.status = "Stop"
            self.t2 = datetime.now()
    def __add__(self,other):
        if self.status == "Start":
            self.t2 = datetime.now()
        if other.status == "Start":
            other.t2 = datetime.now()
        return self.t2 - self.t1 + other.t2 - other.t1
    def __str__(self):  # 当实例化需要作为字符串输出时，调用
        if self.status == "Start":
            self.t2 = datetime.now()
        return str(self.t2 - self.t1)
    __repr__ = __str__  # 当直接输入实例化时，调用

timer1 = Timer()
timer2 = Timer()

In [21]:
timer1.start()

In [22]:
timer1.stop()
timer1  # 对应__str__

0:00:03.492867

In [23]:
timer2.start()

In [24]:
timer2.stop()
print(timer2)  # 对应__repr__
print(timer1+timer2)

0:00:02.226649
0:00:05.719516


In [56]:
# 参考答案
import time

class MyTimer():
    def start(self):
        self.start = time.localtime()
    def stop(self):
        self.stop = time.localtime()
        self._calc()  
    # 私有方法，外部无法调用
    def _calc(self):
        self._unit = ["年","月","日","时","分","秒"]
        self._lasted = []
        self._prompt = "运行时长："
        for i in range(6):
            self._lasted.append(self.stop[i] - self.start[i])
            if self._lasted[i]:  # 不等于0
                self._prompt += (str(self._lasted[i]) + self._unit[i])
        print(self._prompt)

In [57]:
t1 = MyTimer()
t1.start()

In [58]:
t1.stop()

运行时长：1分-35秒


# P46 045：属性访问

In [59]:
# __getattr__(self, name)  # 访问不存在属性时调用
# __getattribute__(self, name)  # 类的属性被访问时调用
# __setattr__(self, name)  # 类的属性被赋值时调用
# __delattr__(self, name)  # 类的属性被删除时调用
# 先调用__getattribute__，如果属性不存在，继续调用__getattr__

In [75]:
class Rectangle:
    def __init__(self, width=0, length=0):
        self.width = width
        self.length = length
    def __setattr__(self, name, value):
        if name == "square":
            self.width = value
            self.length = value
        else:
            # self.name = value  # 触发：无限递归循环
            super().__setattr__(name, value)  # 调用基类的方法，推荐！
            # self.__dict__[name] = value
    def getArea(self):
        return self.width * self.length

In [76]:
re1 = Rectangle(4,5)
re1.getArea()

20

In [77]:
re1.square = 9
re1.getArea()

81

In [74]:
re1.__dict__  # 返回re1.的属性字典

{'width': 9, 'length': 9}

# P47 046：描述符、温度计

In [88]:
# __set__和__setattr__的关系：
# __get__优先级大于__getattr__，而__setattr__大于__set__, __delattr__大于__delete__
# __setattr__是针对类的直属、特有属性的操作处理；
# 1.__set__是针对各种属性的封装，封装完成后，可以被各个类引用为自己的属性
# 2.如果一个类需要多中属性，彼此分开处理，也需要使用__set__
class MyProperty:  # 和原Property没什么不同
    def __init__(self, fget=None, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel      
    def __get__(self, instance, owner):  # instance：实例化后的c；owner：类本身——C
        return self.fget(instance)
    def __set__(self, instance, value):
        self.fset(instance, value)
    def _delete_(self, instance):
        self.fdel(instance)

class C:
    def __init__(self):
        self._x = ""
    def getX(self):
        return self._x
    def setX(self, value):
        self._x = value
    def delX(self):
        del self._x
    x = MyProperty(getX, setX, delX)  # 和原Property没什么不同

In [89]:
c = C()
c.x = "Hello"
c.x

'Hello'

In [96]:
class Celsius:  # 基础
    def __init__(self, value=26):
        self.value = float(value)
    def __get__(self, instance, owner):
        return self.value
    def __set__(self, instance, value):
        self.value = float(value)
      
class Fahrenheit:  # 辅助，不要用__init__，必须形成递归循环
    def __get__(self, instance, owner):  # instance是实例化后的temp，owner是类本身——Temperature
        return instance.cel * 1.8 + 32
    def __set__(self, instance, value):
        instance.cel = (float(value) - 32) / 1.8

class Temperature:  # Temperature类有两套属性，可以对这两套属性分别操作，并能相互调用！
    cel = Celsius()  # 不需要参数，因为__get__、__set__中不需要调用
    fah = Fahrenheit()

In [97]:
tem = Temperature()  # 对应instance
print(tem.cel, tem.fah)
tem.cel = 44
print(tem.cel, tem.fah)
tem.fah = 88
print(tem.cel, tem.fah)

26.0 78.80000000000001
44.0 111.2
31.11111111111111 88.0


# P48 047：定制序列

In [113]:
# 对于不可变的序列（类似tuple），需要重新设定__len__和__getitem__
# 对于可变序列，还需要另外设定__setitem__和__delitem__
class CountList:
    def __init__(self, *args):
        self.values = [x for x in args]  # 数组推导式
        self.count = {}.fromkeys(range(len(self.values)),0)  # 生成字典
    def __len__(self):
        return len(self.values)
    def __getitem__(self, key):
        self.count[key] += 1
        return self.values[key]

In [118]:
c1 = CountList(88, 3, 35, -4, 91)
sum = c1[0] + c1[2]
print(c1[0], sum)
for i in c1.values:  # 直接访问values，并不会计算在内
    print(i)
# for i in range(5):
#     print(c1[i])
c1.count

88 123
88
3
35
-4
91


{0: 2, 1: 0, 2: 1, 3: 0, 4: 0}

# P49 048：迭代器、__iter__、__next__

In [20]:
# 1.通过在类中添加 __iter__函数，向系统说明这是一个可迭代对象。
# 2.通过在类中添加 __next__函数，向系统提供该可迭代对象的迭代算法。
# 3.在代码执行过程中，for循环函数会自动检查系统信息，识别__iter__函数，然后自动调用对应的__next__函数，生成一个迭代器。
# 4.所以在定义一个可迭代类时，一般__iter__函数要与__next__函数成对出现。__iter__函数向系统声明这个类可迭代，__next__定义了具体的迭代器。
# 5.__iter__与__next__两个函数名不可改变，否则系统会不识别。
# 6.__next__函数的return在if判别命令的内部，每次执行__next__函数时，单次判别后直接输出。不满足判别条件时输出迭代终止。
class Fibs:
    def __init__(self, max=100):
        self.a, self.b = 0, 1
        self.max = max
    def __iter__(self):
        return self
    def __next__(self):  # 第一次就进入该方法，不会打印self.a的默认值
        self.a, self.b = self.b, self.a + self.b
        if self.a > self.max:
            raise StopIteration()
        return self.a

In [36]:
for i in Fibs(89):
    print(i, end=" ")

1 1 2 3 5 8 13 21 34 55 89 

# P50 049：yield/生成器、推导式

In [29]:
# yield的使用场景和有点：
# 数据、字典等序列的缺陷是所有数据都在内存中，如果有海量数据的话将会非常耗内存。
# 生成器是可以迭代的，但只可以读取它一次。因为用的时候才生成。
def fibs(max=100):
    a, b = 0, 1
    while a < max:
        a, b = b, a+b
        print(a, end=" ")

In [30]:
fibs()

1 1 2 3 5 8 13 21 34 55 89 144 

In [1]:
def fibs():
    a, b = 0, 1
    while True:
        a, b = b, a+b
        yield a  # 每次执行到此，就回暂停，不会陷入无限循环
        print("在a的之后")  # 每次在yield处中断，next()之后从中断处继续执行

for i in fibs():
    if i < 100:
        print(i, end=" ")
    else:
        break

1 在a的之后
1 在a的之后
2 在a的之后
3 在a的之后
5 在a的之后
8 在a的之后
13 在a的之后
21 在a的之后
34 在a的之后
55 在a的之后
89 在a的之后


In [41]:
li1 = [i for i in range(10) if not (i%2)]  # 数组推导式
print(li)
dict1 = {i:i%2==0 for i in range(10)}  # 字典推导式
print(dict)
set1 = {i for i in range(10) if not (i%2)}  # 集合推导式
print(set1)
tuple1 = (i for i in range(10) if not (i%2))  # 生成器推导式，不是元组推导式
print(tuple1)

[0, 2, 4, 6, 8]
{0: True, 1: False, 2: True, 3: False, 4: True, 5: False, 6: True, 7: False, 8: True, 9: False}
{0, 2, 4, 6, 8}
<generator object <genexpr> at 0x0000000005B981B0>


In [42]:
for i in tuple1:
    print(i, end=" ")

0 2 4 6 8 

In [44]:
n1 = sum(i for i in range(10) if not (i%2))  # 更简洁
n2 = sum((i for i in range(10) if not (i%2)))
print(n1, n2)

20 20


# 52 051：__name__、__init__.py

In [ ]:
# package的文件夹下面必须有一个__init__.py的文件，文件可以为空

In [60]:
class C:
    def test():
        print("这是测试")
    if __name__ == "__main__":  # 只有测试时被调用，本模块被外部import时，不会被调用
        test()

这是测试


In [61]:
print(__name__)
print(C.__name__)  # 是C，不是实例化

__main__
C


In [2]:
import sys
sys.path.append("C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages #最好存入这个路径")
sys.path

['C:\\Users\\Administrator\\AAAPython3',
 'C:\\Users\\Administrator\\Anaconda3\\python37.zip',
 'C:\\Users\\Administrator\\Anaconda3\\DLLs',
 'C:\\Users\\Administrator\\Anaconda3\\lib',
 'C:\\Users\\Administrator\\Anaconda3',
 '',
 'C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Administrator\\.ipython',
 'C:\\Users\\Administrator\\Anaconda3\\lib\\site-packages #最好存入这个路径']